# Titanic Disaster - Classification

In [840]:
#%pip install scikit-learn
#%pip install scikit-optimize

In [841]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

pd.set_option('future.no_silent_downcasting', True)


In [842]:
#%% abrir o datase de treino e teste

train = pd.read_csv('./train.csv')
test  = pd.read_csv('./test.csv')


In [843]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [844]:
train['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [845]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [846]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [847]:
train_test_data = [train,test] # combine dataset

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
dataset

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,Mr
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,Dona
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,Mr


In [848]:
print(test['Cabin'].unique())
print(train['Cabin'].unique())

[nan 'B45' 'E31' 'B57 B59 B63 B66' 'B36' 'A21' 'C78' 'D34' 'D19' 'A9'
 'D15' 'C31' 'C23 C25 C27' 'F G63' 'B61' 'C53' 'D43' 'C130' 'C132' 'C101'
 'C55 C57' 'B71' 'C46' 'C116' 'F' 'A29' 'G6' 'C6' 'C28' 'C51' 'E46' 'C54'
 'C97' 'D22' 'B10' 'F4' 'E45' 'E52' 'D30' 'B58 B60' 'E34' 'C62 C64' 'A11'
 'B11' 'C80' 'F33' 'C85' 'D37' 'C86' 'D21' 'C89' 'F E46' 'A34' 'D' 'B26'
 'C22 C26' 'B69' 'C32' 'B78' 'F E57' 'F2' 'A18' 'C106' 'B51 B53 B55'
 'D10 D12' 'E60' 'E50' 'E39 E41' 'B52 B54 B56' 'C39' 'B24' 'D28' 'B41'
 'C7' 'D40' 'D38' 'C105']
[nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33'
 'B30' 'C52' 'B28' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49'
 'F4' 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77'
 'E67' 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106'
 'C65' 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91'
 'E40' 'T' 'C128' 'D37' 'B35' 'E50' '

In [849]:
def export_prediction(prediction, name):
    output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': prediction})
    output.to_csv(name, index=False)
    print("Your submission was successfully saved!")

In [850]:
def oheInAttr(X, nameAttr):
    encoder = OneHotEncoder(sparse_output=False)
    X_Pclass = X[[nameAttr]]
    # Aplica o encoder e cria um dataframe com as colunas resultantes
    encoded_pclass = pd.DataFrame(encoder.fit_transform(X_Pclass), 
                                  columns=encoder.get_feature_names_out([nameAttr]))
    # Junta as colunas encodadas ao dataframe original
    X = pd.concat([X, encoded_pclass], axis=1)
    # Opcional: Remova a coluna original 'Pclass'
    X.drop(nameAttr, axis=1, inplace=True)
    return X

In [851]:
def predict_export(models, y_test):
    for model in models:        
        #print(f'submission_{model}.csv')
        predictions = model.predict(y_test) 
        export_prediction(predictions, f'submission_{model}.csv')

In [852]:
# Implementando o cross validation

# data and target
data, target = train.drop(columns = ['PassengerId']), train['Survived']
data_test = test.drop(columns = ['PassengerId'])

In [853]:
test['Cabin'].str[:1]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
413    NaN
414      C
415    NaN
416    NaN
417    NaN
Name: Cabin, Length: 418, dtype: object

In [854]:
data['Cabin'] = data['Cabin'].str[:1]
data_test['Cabin'] = data_test['Cabin'].str[:1]

data['Cabin'] = data['Cabin'].fillna('Z')
data_test['Cabin'] = data_test['Cabin'].fillna('Z')

print(data['Cabin'].isna().sum(), test['Cabin'].isna().sum())

0 327


In [855]:
X_train = data.copy()
y_test = data_test.copy()


def features_create(X, usaSurvived):    
    
    print(X['Cabin'].unique())
        
    X['Age'] = X['Age'].fillna(X['Age'].mean())
    
    subs = {'female':1, 'male':0}
    X['mulher'] = X['Sex'].replace(subs)
    
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])
    subs = {'S':1, 'C':2, 'Q':3}
    X['porto'] = X['Embarked'].replace(subs)
    
    X['crianca'] = 1
    X['crianca'] = np.where(X['Age'] < 12, 1, 0)
    
    X['caroOrBarato'] = 0
    #X['caroOrBarato'] = np.where(X['Fare'] > 50, 1, 0)

    X.loc[ X['Fare'] <= 32, 'caroOrBarato'] = int(0)
    X.loc[(X['Fare'] > 32) & (X['Fare'] <= 50), 'caroOrBarato'] = int(1)
    X.loc[(X['Fare'] > 50) & (X['Fare'] <= 100), 'caroOrBarato'] = int(2)
    X.loc[(X['Fare'] > 100) & (X['Fare'] <= 250), 'caroOrBarato'] = int(3)
    X.loc[(X['Fare'] > 250), 'caroOrBarato'] = int(4)
    

    X.loc[ X['Age'] <= 16, 'Age'] = int(0)
    X.loc[(X['Age'] > 16) & (X['Age'] <= 32), 'Age'] = int(1)
    X.loc[(X['Age'] > 32) & (X['Age'] <= 48), 'Age'] = int(2)
    X.loc[(X['Age'] > 48) & (X['Age'] <= 64), 'Age'] = int(3)
    X.loc[ X['Age'] > 64, 'Age'] = int(4)
    
    # mulher com filho
    X["mulherComEsposo"] = 0
    X.loc[(X['mulher'] == 1) & (X['SibSp'] >0), 'mulherComEsposo'] = 1
    
    X["mulherComFilho"] = 0
    X.loc[(X['mulher'] == 1) & (X['Parch'] > 0), 'mulherComFilho'] = 1
    
    X["mulherComEsposoFilho"] = 0
    X.loc[(X['mulher'] == 1) & (X['SibSp'] > 0) & (X['Parch'] > 0), 'mulherComEsposoFilho'] = 1
    
    X["homemComEsposa"] = 0
    X.loc[(X['mulher'] == 0) & (X['SibSp'] >0), 'homemComEsposa'] = 1
    
    X["homemComFilho"] = 0
    X.loc[(X['mulher'] == 0) & (X['Parch'] > 0), 'homemComFilho'] = 1
    
    X["homemComEsposaFilho"] = 0
    X.loc[(X['mulher'] == 0) & (X['SibSp'] > 0) & (X['Parch'] > 0), 'homemComEsposaFilho'] = 1

    
    X['Title'] = X['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, 
                    "Master": 3, "Dr": 3, "Rev": 3, "Col": 3, "Major": 3, "Mlle": 3,"Countess": 3,
                    "Ms": 3, "Lady": 3, "Jonkheer": 3, "Don": 3, "Dona" : 3, "Mme": 3,"Capt": 3,"Sir": 3 }

    X['Title'] = X["Title"].map(title_mapping)
    
    cabin_mapping = {"Z": 0, "A": 0.2, "B": 0.4, "C": 0.8, "D": 1.2, "E": 1.6, "F": 2, "G": 2.4, "T": 2.8}
    X['Cabin'] = X['Cabin'].map(cabin_mapping)
    X["Cabin"] = X["Cabin"].fillna(X.groupby(["Pclass"])["Cabin"].transform("median"))
    
    
    # X = oheInAttr(X, "Sex")
    # X = oheInAttr(X, "Embarked")    
    # X = oheInAttr(X, "Pclass")
    # X = oheInAttr(X, "Age")
    # X = oheInAttr(X, "Title")
    # X = oheInAttr(X, "porto")
        
    
    #if (usaSurvived):        
    # X["resultado"] = 0 #Survived
    # X.loc[(X['mulher'] == 1) & (X['mulherComEsposo'] == 1) & (X['Survived'] == 1), 'resultado'] = 1
    # X.loc[(X['mulher'] == 1) & (X['mulherComEsposoFilho'] == 1) & (X['Survived'] == 1), 'resultado'] = 1
    # X.loc[(X['mulher'] == 1) & (X['mulherComFilho'] == 1) & (X['Survived'] == 1), 'resultado'] = 1
    
    # X.loc[(X['mulher'] == 0) & (X['homemComEsposa'] == 1) & (X['Survived'] == 1), 'resultado'] = 1
    # X.loc[(X['mulher'] == 0) & (X['homemComFilho'] == 1) & (X['Survived'] == 1), 'resultado'] = 1
    # X.loc[(X['mulher'] == 0) & (X['homemComEsposaFilho'] == 1) & (X['Survived'] == 1), 'resultado'] = 1
    
    
    
    featureDrop = ["Name", "Ticket", 'Embarked', "Sex"]
    X = X.drop(columns = featureDrop, axis=0)
    
    
    return X


X_train = features_create(X_train, True)
y_test  = features_create(y_test, False)

# Remove a coluna Fare
feature_remove = ['Fare','SibSp', 'Parch']
feature_remove2 = feature_remove + ['Survived']

X_train = X_train.drop(columns = feature_remove2, axis=0)
y_test = y_test.drop(columns = feature_remove, axis=0)

# Teste com scalar
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
y_test_sc = scaler.fit_transform(y_test)

print(X_train['Cabin'].unique())
print(y_test['Cabin'].unique())


['Z' 'C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']
['Z' 'B' 'E' 'A' 'C' 'D' 'F' 'G']
[0.  0.8 1.6 2.4 1.2 0.2 0.4 2.  2.8]
[0.  0.4 1.6 0.2 0.8 1.2 2.  2.4]


In [856]:
y_test.describe()

,Pclass,Age,Cabin,Title,crianca,caroOrBarato,mulherComEsposo,mulherComFilho,mulherComEsposoFilho,homemComEsposa,homemComFilho,homemComEsposaFilho
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,1.303828,0.203349,0.732057,0.055024,0.523923,0.153110,0.126794,0.078947,0.169856,0.098086,0.074163
std,0.841838,0.762449,0.458847,0.972019,0.228300,1.022834,0.360525,0.333141,0.269980,0.375957,0.297787,0.262349
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,4.000000,2.400000,3.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [857]:
X_train.columns

Index(['Pclass', 'Age', 'Cabin', 'Title', 'mulher', 'porto', 'crianca',
       'caroOrBarato', 'mulherComEsposo', 'mulherComFilho',
       'mulherComEsposoFilho', 'homemComEsposa', 'homemComFilho',
       'homemComEsposaFilho'],
      dtype='object')

In [858]:
X_train

,Pclass,Age,Cabin,Title,mulher,porto,crianca,caroOrBarato,mulherComEsposo,mulherComFilho,mulherComEsposoFilho,homemComEsposa,homemComFilho,homemComEsposaFilho
0,3,1.0,0.0,0,0,1,0,0,0,0,0,1,0,0
1,1,2.0,0.8,2,1,2,0,2,1,0,0,0,0,0
2,3,1.0,0.0,1,1,1,0,0,0,0,0,0,0,0
3,1,2.0,0.8,2,1,1,0,2,1,0,0,0,0,0
4,3,2.0,0.0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,1.0,0.0,3,0,1,0,0,0,0,0,0,0,0
887,1,1.0,0.4,1,1,1,0,0,0,0,0,0,0,0
888,3,1.0,0.0,1,1,1,0,0,1,1,1,0,0,0
889,1,1.0,0.8,0,0,2,0,0,0,0,0,0,0,0


In [859]:
# print(X_train["mulherComEsposo"].value_counts())
# print(X_train["mulherComFilho"].value_counts())
# print(X_train["mulherComEsposoFilho"].value_counts())


In [860]:
print(X_train.info())

print()

#verificar valores nulos ou NAN
print(X_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Pclass                891 non-null    int64  
 1   Age                   891 non-null    float64
 2   Cabin                 891 non-null    float64
 3   Title                 891 non-null    int64  
 4   mulher                891 non-null    object 
 5   porto                 891 non-null    object 
 6   crianca               891 non-null    int64  
 7   caroOrBarato          891 non-null    int64  
 8   mulherComEsposo       891 non-null    int64  
 9   mulherComFilho        891 non-null    int64  
 10  mulherComEsposoFilho  891 non-null    int64  
 11  homemComEsposa        891 non-null    int64  
 12  homemComFilho         891 non-null    int64  
 13  homemComEsposaFilho   891 non-null    int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 97.6+ KB
None

Pclass   

In [861]:
print(y_test.info())

print()

#verificar valores nulos ou NAN
print(y_test.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Pclass                418 non-null    int64  
 1   Age                   418 non-null    float64
 2   Cabin                 418 non-null    float64
 3   Title                 418 non-null    int64  
 4   mulher                418 non-null    object 
 5   porto                 418 non-null    object 
 6   crianca               418 non-null    int64  
 7   caroOrBarato          418 non-null    int64  
 8   mulherComEsposo       418 non-null    int64  
 9   mulherComFilho        418 non-null    int64  
 10  mulherComEsposoFilho  418 non-null    int64  
 11  homemComEsposa        418 non-null    int64  
 12  homemComFilho         418 non-null    int64  
 13  homemComEsposaFilho   418 non-null    int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 45.8+ KB
None

Pclass   

In [862]:
X_train.describe()

,Pclass,Age,Cabin,Title,crianca,caroOrBarato,mulherComEsposo,mulherComFilho,mulherComEsposoFilho,homemComEsposa,homemComFilho,homemComEsposaFilho
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,1.290685,0.222447,0.710438,0.076319,0.485971,0.157127,0.134680,0.080808,0.160494,0.104377,0.078563
std,0.836071,0.812620,0.490484,0.968778,0.265657,0.953158,0.364125,0.341573,0.272693,0.367270,0.305921,0.269207
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,4.000000,2.800000,3.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [863]:
y_test.describe()

,Pclass,Age,Cabin,Title,crianca,caroOrBarato,mulherComEsposo,mulherComFilho,mulherComEsposoFilho,homemComEsposa,homemComFilho,homemComEsposaFilho
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,1.303828,0.203349,0.732057,0.055024,0.523923,0.153110,0.126794,0.078947,0.169856,0.098086,0.074163
std,0.841838,0.762449,0.458847,0.972019,0.228300,1.022834,0.360525,0.333141,0.269980,0.375957,0.297787,0.262349
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,4.000000,2.400000,3.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [864]:
# KNN classifier model
knn = KNeighborsClassifier()

# K-fold (k=5)
scores = cross_val_score(knn, X_train, target, cv=5, scoring='accuracy')

print(scores.mean())

# Results
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

knn.fit(X_train, target)

predictions = knn.predict(X_train) 
#predictions = knn.predict(X_train_sc) 


print(confusion_matrix(target, predictions))

print() 

accuracy = accuracy_score(target, predictions)
print("Accuracy Score:", accuracy)


predictions_test = knn.predict(y_test) 


# predictions = knn.predict(y_test) 
# print(y_test.sum())
# print(predictions)

# Resultado com oneHotEncoder
# 0.6981357102504551
# Accuracy: 0.70 (+/- 0.05)

# Resultado com atribuição
# 0.6835791852363317
# Accuracy: 0.68 (+/- 0.09)

# Sem a coluna Fare e inclusão da criança
# 0.7677044755508129
# Accuracy: 0.77 (+/- 0.05)

export_prediction(predictions_test, "submission_knn_3.csv")

0.7889900194589166
Accuracy: 0.79 (+/- 0.04)
[[491  58]
 [ 77 265]]

Accuracy Score: 0.8484848484848485
Your submission was successfully saved!


In [865]:
X_train

,Pclass,Age,Cabin,Title,mulher,porto,crianca,caroOrBarato,mulherComEsposo,mulherComFilho,mulherComEsposoFilho,homemComEsposa,homemComFilho,homemComEsposaFilho
0,3,1.0,0.0,0,0,1,0,0,0,0,0,1,0,0
1,1,2.0,0.8,2,1,2,0,2,1,0,0,0,0,0
2,3,1.0,0.0,1,1,1,0,0,0,0,0,0,0,0
3,1,2.0,0.8,2,1,1,0,2,1,0,0,0,0,0
4,3,2.0,0.0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,1.0,0.0,3,0,1,0,0,0,0,0,0,0,0
887,1,1.0,0.4,1,1,1,0,0,0,0,0,0,0,0
888,3,1.0,0.0,1,1,1,0,0,1,1,1,0,0,0
889,1,1.0,0.8,0,0,2,0,0,0,0,0,0,0,0


In [866]:
# X = pd.get_dummies(X_train[features])
# X_test = pd.get_dummies(X_train[features])

model_rfc = RandomForestClassifier(criterion = 'entropy', 
                                  n_estimators = 500, 
                                  max_depth = 5, 
                                  min_samples_split = 3, 
                                  min_samples_leaf = 1, 
                                  random_state = 0)

# model_rfc = RandomForestClassifier(criterion = 'entropy', 
#                                   n_estimators = 500, 
#                                   max_depth = 5, 
#                                   min_samples_split = 5, 
#                                   min_samples_leaf = 2, 
#                                   random_state = 0)


# model_rfc = RandomForestClassifier(criterion = otimos.x[0], 
#                                   n_estimators = otimos.x[1], 
#                                   max_depth = otimos.x[2], 
#                                   min_samples_split = otimos.x[3], 
#                                   min_samples_leaf = otimos.x[4], 
#                                   random_state = 0, 
#                                   n_jobs = -1 )

#model_rfc = RandomForestClassifier()

# K-fold (k=5)
scores = cross_val_score(model_rfc, X_train, target, cv=5, scoring='accuracy')

print(scores.mean())

# Results
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

model_rfc.fit(X_train, target)

predictions = model_rfc.predict(X_train) 
#predictions = model_rfc.predict(X_train_sc) 


print(confusion_matrix(target, predictions))

print() 

accuracy = accuracy_score(target, predictions)
print("Accuracy Score:", accuracy)


predictions_test = model_rfc.predict(y_test) 

# print(y_test.sum())
# print(predictions)



#print(accuracy_score(y_test, predictions))

# Result
# 0.8215805661917018
# Accuracy: 0.82 (+/- 0.06)

# print(np.mean(score))

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
# model.fit(X, y)

# predictions = model.predict(X_test)

# print(predictions)

export_prediction(predictions_test, "submission_rfc_2.csv")

0.8136902893729208
Accuracy: 0.81 (+/- 0.04)
[[498  51]
 [ 97 245]]

Accuracy Score: 0.8338945005611672
Your submission was successfully saved!


In [875]:
# Melhor accuracy
#learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier


clf = [KNeighborsClassifier(n_neighbors = 13), 
       DecisionTreeClassifier(),
       RandomForestClassifier(n_estimators=13), 
       GaussianNB(), 
       SVC(kernel='linear', C=1), 
       ExtraTreeClassifier(),
       GradientBoostingClassifier(n_estimators=10, learning_rate=1,max_features=3, max_depth =3, random_state = 10),
       AdaBoostClassifier(),
       ExtraTreesClassifier()]

k_fold = 5

def model_fit():
    models =[]
    scoring = 'accuracy'
    for i in range(len(clf)):
        print("Model Training ", clf[i])
        score = cross_val_score(clf[i], X_train, target, cv=k_fold, n_jobs=1, scoring=scoring)
        print("Score of Model",i,":",round(np.mean(score)*100,2))
        models.append(clf[i].fit(X_train, target))
        predictions = clf[i].predict(X_train) 
        print("Confusion Matrix:")
        print(confusion_matrix(target, predictions))
        accuracy = accuracy_score(target, predictions)
        print("Accuracy Score:", accuracy)
        print() 
#     round(np.mean(score)*100,2)
#     print("Score of :\n",score)
    return models

models = model_fit()

Model Training  KNeighborsClassifier(n_neighbors=13)
Score of Model 0 : 78.68
Confusion Matrix:
[[481  68]
 [ 92 250]]
Accuracy Score: 0.8204264870931538

Model Training  DecisionTreeClassifier()
Score of Model 1 : 79.8
Confusion Matrix:
[[529  20]
 [ 76 266]]
Accuracy Score: 0.8922558922558923

Model Training  RandomForestClassifier(n_estimators=13)
Score of Model 2 : 80.81
Confusion Matrix:
[[517  32]
 [ 64 278]]
Accuracy Score: 0.8922558922558923

Model Training  GaussianNB()
Score of Model 3 : 75.2
Confusion Matrix:
[[462  87]
 [126 216]]
Accuracy Score: 0.7609427609427609

Model Training  SVC()
Score of Model 4 : 80.69
Confusion Matrix:
[[499  50]
 [103 239]]
Accuracy Score: 0.8282828282828283

Model Training  ExtraTreeClassifier()
Score of Model 5 : 78.45
Confusion Matrix:
[[529  20]
 [ 76 266]]
Accuracy Score: 0.8922558922558923

Model Training  GradientBoostingClassifier(learning_rate=1, max_features=3, n_estimators=10,
                           random_state=10)
Score of Model

/home/romildopaiter/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/romildopaiter/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/romildopaiter/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/romildopaiter/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated a

Score of Model 7 : 79.58
Confusion Matrix:
[[473  76]
 [ 84 258]]
Accuracy Score: 0.8204264870931538

Model Training  ExtraTreesClassifier()
Score of Model 8 : 79.91
Confusion Matrix:
[[529  20]
 [ 76 266]]
Accuracy Score: 0.8922558922558923



In [876]:
predict_export(models, y_test=y_test)

Your submission was successfully saved!
Your submission was successfully saved!
Your submission was successfully saved!
Your submission was successfully saved!
Your submission was successfully saved!
Your submission was successfully saved!
Your submission was successfully saved!
Your submission was successfully saved!
Your submission was successfully saved!


In [869]:
# X = pd.get_dummies(X_train[features])
# X_test = pd.get_dummies(X_train[features])

model_gbc = GradientBoostingClassifier(n_estimators=10, 
                                       learning_rate=1,
                                       max_features=3, 
                                       max_depth =3, 
                                       random_state = 10)

# K-fold (k=5)
scores = cross_val_score(model_gbc, X_train, target, cv=5, scoring='accuracy')

print(scores.mean())

# Results
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

model_gbc.fit(X_train, target)

predictions = model_gbc.predict(X_train) 
#predictions = model_rfc.predict(X_train_sc) 


print(confusion_matrix(target, predictions))

print() 

accuracy = accuracy_score(target, predictions)
print("Accuracy Score:", accuracy)


predictions_test = model_gbc.predict(y_test) 

# print(y_test.sum())
# print(predictions)



#print(accuracy_score(y_test, predictions))

# Result
# 0.8215805661917018
# Accuracy: 0.82 (+/- 0.06)

# print(np.mean(score))

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
# model.fit(X, y)

# predictions = model.predict(X_test)

# print(predictions)

export_prediction(predictions_test, "submission_model_gbc_1.csv")

0.8114807607808675
Accuracy: 0.81 (+/- 0.06)
[[491  58]
 [ 79 263]]

Accuracy Score: 0.8462401795735129
Your submission was successfully saved!


In [870]:
from skopt import gp_minimize

def treinar_modelo(parametros):
  
  model_rf = RandomForestClassifier(criterion = parametros[0], n_estimators = parametros[1], max_depth = parametros[2], 
                                    min_samples_split = parametros[3], min_samples_leaf = parametros[4], 
                                    random_state = 0, 
                                    n_jobs = -1 )
  
  score = cross_val_score(model_rf, X_train, target, cv = 10)
  mean_score = np.mean(score)
  print(np.mean(score))
  return -mean_score

parametros = [('entropy', 'gini'), 
              (500, 1000),
              (3, 20),
              (2, 10),
              (1, 5)]

otimos = gp_minimize(treinar_modelo, parametros, random_state = 0, verbose = 1, n_calls = 30, n_random_starts = 10  )


print(otimos.fun, otimos.x)

Iteration No: 1 started. Evaluating function at random point.
0.812621722846442
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 13.6426
Function value obtained: -0.8126
Current minimum: -0.8126
Iteration No: 2 started. Evaluating function at random point.
0.8204244694132334
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 10.3443
Function value obtained: -0.8204
Current minimum: -0.8204
Iteration No: 3 started. Evaluating function at random point.
0.8148689138576779
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 11.6942
Function value obtained: -0.8149
Current minimum: -0.8204
Iteration No: 4 started. Evaluating function at random point.
0.8103745318352059
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 10.7374
Function value obtained: -0.8104
Current minimum: -0.8204
Iteration No: 5 started. Evaluating function at random point.
0.8070037453183521
Iteration No: 5 ended. Evaluation done at random point.
Time

KeyboardInterrupt: 